In [1]:
############## MAKING ALL NECESSARY IMPORTS #################

In [1]:
import warnings
warnings.filterwarnings("ignore")
import csv
import math
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import json
import nltk
import string
import urllib
from pprint import pprint
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize 
from nltk.wsd import lesk
#from nltk.parse import CoreNLPParser
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
#from nltk.parse.corenlp import CoreNLPDependencyParser

In [ ]:
############## MOVIE MATCHING BETWEEN DATASETS #############################

In [2]:
df_movies_summaries = pd.read_csv(r'Data/MovieSummaries/movie.metadata.tsv', sep='\t', header=None)
df_movies_summaries.columns = ['Wikipedia ID', 'Freebase ID', 'Movie name', 'Release date', 'BO Revenue', 'Runtime', 'Language', 'Countires', 'Genre']

In [4]:
with open('Data/ml-1m/movies.dat') as dat_file, open('Data/ml-1m/movies.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        row = [field.strip() for field in line.split('::')]
        csv_writer.writerow(row)

df_movies_ratings = pd.read_csv('Data/ml-1m/movies.csv', sep=',', encoding='latin-1')
df_movies_ratings.columns = ['movie_id', 'title', 'genre']

In [5]:
df_movies_ratings['title'] = df_movies_ratings['title'].str.lower()
df_movies_summaries['Movie name'] = df_movies_summaries['Movie name'].str.lower()

In [6]:
movie_ratings_dict = {}
for ix in df_movies_ratings.index:
    movie_id = df_movies_ratings.loc[ix]['movie_id']
    movie_name = df_movies_ratings.loc[ix]['title']
    temp_list = movie_name.split('(')
    movie_name_new = temp_list[0]
    movie_name_new = movie_name_new.replace(' ', '')
    movie_ratings_dict[movie_name_new] = movie_id

In [7]:
movie_id_ratings_summary = {}
for ix in df_movies_summaries.index:
    movie_name = df_movies_summaries.loc[ix]['Movie name']
    movie_id_summary = df_movies_summaries.loc[ix]['Wikipedia ID']
    if movie_name in movie_ratings_dict:
        df_movies_summaries.loc[[ix],['Ratings present']] = 'Y'
        movie_id_rating = movie_ratings_dict[movie_name]
        df_movies_summaries.loc[[ix],['Movie ID Ratings']] = movie_id_rating
        movie_id_ratings_summary[movie_id_rating] = movie_id_summary

In [8]:
df_movies_summaries.to_csv('Output/df_movies_summaries.csv')
with open('Output/movie-id-ratings-summary.txt', 'w') as f:
    print(movie_id_ratings_summary, file=f)


In [ ]:
print(df_movies_summaries.nunique())
print(len(movie_id_ratings_summary))

In [10]:
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
rake = Rake()

In [ ]:
# FOR TESTING
# text = "The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker's son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the career tributes who train intensively at special academies and almost always win. During a TV interview with Caesar Flickerman, Peeta unexpectedly reveals his love for Katniss. She is outraged, believing it to be a ploy to gain audience support, as sponsors may provide in-Games gifts of food, medicine, and tools. However, she discovers Peeta meant what he said. The televised Games begin with half of the tributes killed in the first few minutes; Katniss barely survives ignoring Haymitch's advice to run away from the melee over the tempting supplies and weapons strewn in front of a structure called the Cornucopia. Peeta forms an uneasy alliance with the four Careers. They later find Katniss and corner her up a tree. Rue, hiding in a nearby tree, draws her attention to a poisonous tracker jacker nest hanging from a branch. Katniss drops it on her sleeping besiegers. They all scatter, except for Glimmer, who is killed by the insects. Hallucinating due to tracker jacker venom, Katniss is warned to run away by Peeta. Rue cares for Katniss for a couple of days until she recovers. Meanwhile, the alliance has gathered all the supplies into a pile. Katniss has Rue draw them off, then destroys the stockpile by setting off the mines planted around it. Furious, Cato kills the boy assigned to guard it. As Katniss runs from the scene, she hears Rue calling her name. She finds Rue trapped and releases her. Marvel, a tribute from District 1, throws a spear at Katniss, but she dodges the spear, causing it to stab Rue in the stomach instead. Katniss shoots him dead with an arrow. She then comforts the dying Rue with a song. Afterward, she gathers and arranges flowers around Rue's body. When this is televised, it sparks a riot in Rue's District 11. President Snow summons Seneca Crane, the Gamemaker, to express his displeasure at the way the Games are turning out. Since Katniss and Peeta have been presented to the public as star-crossed lovers, Haymitch is able to convince Crane to make a rule change to avoid inciting further riots. It is announced that tributes from the same district can win as a pair. Upon hearing this, Katniss searches for Peeta and finds him with an infected sword wound in the leg. She portrays herself as deeply in love with him and gains a sponsor's gift of soup. An announcer proclaims a feast, where the thing each survivor needs most will be provided. Peeta begs her not to risk getting him medicine. Katniss promises not to go, but after he falls asleep, she heads to the feast. Clove ambushes her and pins her down. As Clove gloats, Thresh, the other District 11 tribute, kills Clove after overhearing her tormenting Katniss about killing Rue. He spares Katniss just this time...for Rue. The medicine works, keeping Peeta mobile. Foxface, the girl from District 5, dies from eating nightlock berries she stole from Peeta; neither knew they are highly poisonous. Crane changes the time of day in the arena to late at night and unleashes a pack of hound-like creatures to speed things up. They kill Thresh and force Katniss and Peeta to flee to the roof of the Cornucopia, where they encounter Cato. After a battle, Katniss wounds Cato with an arrow and Peeta hurls him to the creatures below. Katniss shoots Cato to spare him a prolonged death. With Peeta and Katniss apparently victorious, the rule change allowing two winners is suddenly revoked. Peeta tells Katniss to shoot him. Instead, she gives him half of the nightlock. However, before they can commit suicide, they are hastily proclaimed the victors of the 74th Hunger Games. Haymitch warns Katniss that she has made powerful enemies after her display of defiance. She and Peeta return to District 12, while Crane is locked in a room with a bowl of nightlock berries, and President Snow considers the situation."
# rake.extract_keywords_from_text(text)
# key_words_dict_scores = rake.get_word_degrees()
# print(key_words_dict_scores)

In [11]:
# filename = "Data/MovieSummaries/plot_summaries.txt"
# counts_dict = {}
# movies_dict = {}
# df_movies_concepts = pd.DataFrame()
# data = []

# with open(filename, encoding="utf8") as file:
#     for line in file:
#         hypernyms_list = []
#         values = line.split("\t")
#         synopsis = values[1]
#         movieID = values[0]
#         if movieID in str(movie_id_ratings_summary.values()):
#             rake.extract_keywords_from_text(synopsis)
#             key_words_dict_scores = rake.get_word_degrees()
#             keywords_list = list(key_words_dict_scores.keys())
#             keywords_string = ' '.join([str(item) for item in keywords_list])
#             data.append([movieID, keywords_string])
#             movies_dict[movieID] = keywords_list

In [12]:
# df_movies_concepts = pd.DataFrame(data, columns=['Movie ID (Summary)', 'Keywords List'])

In [13]:
# df_movies_concepts.to_csv('Output/df_movies_concepts.csv')
#print(len(movies_dict))

In [15]:
# count = CountVectorizer()
# count_matrix = count.fit_transform(df_movies_concepts['Keywords List'])
# cosine_sim_matrix = cosine_similarity(count_matrix, count_matrix)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
  
  
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
  
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
  
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}

In [51]:
filename = "Data/MovieSummaries/plot_summaries.txt"
counts_dict = {}
movies_dict = {}
df_movies_concepts = pd.DataFrame()
data = []

with open(filename, encoding="utf8") as file:
    for line in file:
        values = line.split("\t")
        synopsis = values[1]
        movieID = values[0]
        if movieID in str(movie_id_ratings_summary.values()):
            data.append([movieID, synopsis])
            movies_dict[movieID] = synopsis

In [52]:
df_movies_concepts = pd.DataFrame(data, columns=['Movie ID (Summary)', 'Summary'])

In [53]:
def preprocess_sentences(text):
  text = text.lower()
  temp_sent =[]
  words = nltk.word_tokenize(text)
  tags = nltk.pos_tag(words)
  for i, word in enumerate(words):
      if tags[i][1] in VERB_CODES: 
          lemmatized = lemmatizer.lemmatize(word, 'v')
      else:
          lemmatized = lemmatizer.lemmatize(word)
      if lemmatized not in stop_words and lemmatized.isalpha():
          temp_sent.append(lemmatized)
          
  finalsent = ' '.join(temp_sent)
  finalsent = finalsent.replace("n't", " not")
  finalsent = finalsent.replace("'m", " am")
  finalsent = finalsent.replace("'s", " is")
  finalsent = finalsent.replace("'re", " are")
  finalsent = finalsent.replace("'ll", " will")
  finalsent = finalsent.replace("'ve", " have")
  finalsent = finalsent.replace("'d", " would")
  return finalsent
  
df_movies_concepts["Summary Processed"]= df_movies_concepts["Summary"].apply(preprocess_sentences)

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
  
# Vectorizing pre-processed movie plots using TF-IDF
tfidfvec = TfidfVectorizer()
tfidf_movieid = tfidfvec.fit_transform((df_movies_concepts["Summary Processed"]))
  
# Finding cosine similarity between vectors
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim_matrix = cosine_similarity(tfidf_movieid, tfidf_movieid)

In [55]:
with open('Output/cosine-sim-matrix-2.txt', 'w', encoding='utf8') as f:
    print(cosine_sim_matrix, file=f)

In [42]:
total = 0
count = 0
for i in range(len(cosine_sim_matrix)):
    for j in range(len(cosine_sim_matrix[0])):
        total += cosine_sim_matrix[i][j]
        count += 1

average = total / count
print(average)

0.01703050798710823


In [43]:
#print(len(movies_dict))
with open('Output/movies-dict.txt', 'w', encoding='utf8') as f:
    print(movies_dict, file=f)

In [ ]:
################### GETTING THE RATINGS DATA #################################

In [20]:
users_dict = dict()
with open('Data/ml-1m/ratings.dat', encoding='utf8') as dat_file, open('Data/ml-1m/ratings.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        row = [field.strip() for field in line.split('::')]
        csv_writer.writerow(row)

df_ratings = pd.read_csv('Data/ml-1m/ratings.csv', sep=',', encoding='latin-1')
df_ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

In [21]:
print(len(df_ratings))

1000208


In [287]:
# FOR TESTING
# df_ratings = df_ratings[0:40000]
# print(len(df_ratings))

In [ ]:
########################### GENERATING A USER RATING MATRIX FOR REVIEWS OF MOVIES THAT HAVE SUMMARY INFORMATION ######################

In [ ]:
entries = []
ratings_new_list = []
for ix in df_ratings.index:
    movie_id = df_ratings.loc[ix]['movie_id']
    if float(movie_id) in movie_id_ratings_summary:
        df_ratings.loc[[ix],['Summary Present']] = 'Y'
        df_ratings.loc[[ix],['Movie Summary ID']] = int(movie_id_ratings_summary[float(movie_id)])    

In [ ]:
df_ratings = df_ratings[(df_ratings['Summary Present'] == 'Y')]

In [24]:
df_ratings, df_ratings_test = train_test_split(df_ratings, test_size=0.2)

In [23]:
# FOR TESTING
# df_ratings = pd.read_csv('Output/df_ratings.csv')
# df_ratings_test = pd.read_csv('Output/df_ratings_test.csv')

In [25]:
df_ratings_grouped = df_ratings.groupby('user_id')

In [26]:
print(len(df_ratings_grouped))

5985


In [27]:
ratings_matrix = np.zeros(shape=(len(df_ratings_grouped), len(movies_dict)))
key = 0
row = -1
existing_users = []
for ix in df_ratings.index:
    user_id = df_ratings.loc[ix]['user_id']
    if user_id not in existing_users:
        existing_users.append(user_id)
        row = row + 1
        #print(row)
        users_dict[user_id] = row

    movie_id = df_ratings.loc[ix]['movie_id']
    ratings_val = df_ratings.loc[ix]['rating']
    #print(type(movie_id))

    if float(movie_id) in movie_id_ratings_summary:
        movie_id_summary = movie_id_ratings_summary[movie_id]
        #print(type(movie_id_summary))
        if str(movie_id_summary) in movies_dict:
            #print('yes')
            col = list(movies_dict).index(str(movie_id_summary))
            ratings_matrix[row][col] = ratings_val

In [ ]:
with open('Output/users-dict.txt', 'w') as f:
    print(users_dict, file=f)

In [ ]:
with open('Output/ratings-matrix.txt', 'w') as f:
    print(ratings_matrix, file=f)

In [109]:
# FOR TESTING
# ratings_matrix = np.zeros(shape=(len(df_ratings_grouped), len(movies_dict)))
# f = open('ratings-matrix.txt', 'r')
# row_num = 0
# col_num = 0
# for line in f:
#     for i in range(len(line)):
#         if line[i] == '0' or line[i] == '1' or line[i] == '2' or line[i] == '3' or line[i] == '4' or line[i] == '5':
#             ratings_matrix[row_num][col_num] = int(line[i])
#             col_num = col_num + 1
#         if line[i] == ']':
#             row_num = row_num + 1
#             col_num = 0 

In [57]:
def ratings_prediction(user_id, movie_id):
    movie_id = str(movie_id)
    #print(type(movie_id))
    movie_flag = False

    # if movie_id not in movies_dict.keys():
    #     return 0
    for key, value in movies_dict.items():
        if movie_id == key:
            movie_flag = True
    
    if not movie_flag:
        print("Error 1")
        return 0
    if user_id not in users_dict:
        print("Error 2")
        return 0
    
    pred_rating = 0
    count = 0

    user_ix = list(users_dict).index(user_id)

    for key, value in movies_dict.items():
        if key == movie_id:
            continue
        else:
            movie_ix = list(movies_dict).index(key)
            #movie_sim = cosine_sim(key, movie_id)
            movie_num_1 = list(movies_dict).index(key)
            movie_num_2 = list(movies_dict).index(movie_id)
            movie_sim = cosine_sim_matrix[movie_num_1][movie_num_2]
            if movie_sim > average:
                if ratings_matrix[user_ix][movie_ix] != 0:
                    count = count + movie_sim
                    pred_rating = pred_rating + movie_sim * ratings_matrix[user_ix][movie_ix]
    
    if count != 0:
        return pred_rating / count
        print('Error 4')
    else:
        return 0
        print('Error 3')

In [ ]:
pred_ratings = []
count = 0
for ix in df_ratings_test.index:
    movie_id = df_ratings_test.loc[ix]['Movie Summary ID']
    movie_id = str(int(movie_id))
    user_id = df_ratings_test.loc[ix]['user_id']
    pred_rating = ratings_prediction(user_id, movie_id)
    pred_ratings.append(pred_rating)
    count = count + 1

In [59]:
df_ratings_test['Predicted Ratings 3'] = pred_ratings

In [60]:
actual_rating_list = []
pred_rating_list = []
#c = 0
for ix in df_ratings_test.index:
    #c = c + 1
    actual_rating = df_ratings_test.loc[ix]['rating']
    pred_rating = df_ratings_test.loc[ix]['Predicted Ratings 3']
    if pred_rating != 0:
        actual_rating_list.append(actual_rating)
        pred_rating_list.append(pred_rating)

In [61]:
df_ratings.to_csv('Output/df_ratings.csv')
df_ratings_test.to_csv('Output/df_ratings_test.csv')

In [ ]:
correct = 0
total = 0
for i in range(len(actual_rating_list)):
    if actual_rating_list[i] == round(pred_rating_list[i]):
        correct = correct + 1
    total = total + 1

average = correct/total
print(average)

In [ ]:
print(pred_rating_list)